# Deep Learning Models From Scratch

The deep learning models you’ve built so far have used transfer learning.  Transfer learning is great because it allows you to quickly build accurate models with relatively little data.  

But transfer learning isn’t necessary on large datasets, and it only works when your use case has similar visual patterns to the the data used in the pre-trained model.  For example, a pretrained model based on everyday photos won’t work well for applications to satellite photography or for medical images from an MRI machine.

So, if you want to be a versatile with deep learning, you’ll need to learn to build models from scratch.  Fortunately, the code for this is almost identical to the code you used with transfer learning.

I’ll show you an example based on the popular MNIST dataset, which contains handwritten digits. Here's a sample of the images from the MNIST dataset.

<img src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png" alt="MNIST sample images">

After this lesson, you’ll build a model to identify different types of clothing from low resolution images using a dataset called MNIST for Fashion.

Both datasets contain 28 by 28 pixel images. We can load them from a CSV file containing all the data for all the images. This will be your first time loading all your image data from a single file, rather than using `ImageDataGenerator` to load many separate image files. Here’s how we loaded the MNIST data.

To store all the images in a single CSV, each image is represented as a row of the CSV. The first column is the image label. In this data, the label says what digit is shown in each row of the CSV. Was it a 0, 1, 2, 3, etc., all the way up to 9.   The rest of the columns represent pixel intensities.  The pixels are represented in the CSV as a row of 784 pixel intensities, even though the images started as a 28x28 two dimensional grid. 

We’ll write a function `data_prep` to extract the labels and reshape the pixel intensity data back to a 28x28 grid before applying our model.

In [ ]:
import pandas as pd
from tensorflow import keras

img_rows, img_cols = 28, 28
num_classes = 10

def data_prep(raw):
    """ Convert image data from a CSV file as a DataFrame to Numpy arrays """
    out_y = keras.utils.to_categorical(raw.label, num_classes)

    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    out_x = x_shaped_array / 255
    return out_x, out_y

Keras has a `to_categorical` function. We supply the target variable, `raw.label`, as well as the number of different values.  The `to_categorical` function returns a one-hot-encoded version of the target. It represents the ten possible values of the target with ten separate binary columns.  If this is new to you, we have a tutorial on one-hot encoding of categorical features in the intermediate machine learning mini-course. In any case, this gives us the target in the format Keras expects it.

Now we’ll work on the pixel intensities.  We get the data as a numpy array from `raw.values`, then use indexing to take everything after the first column, because the first column was the label.

We’ll reshape this into a 4D array, which is indexed by the image number, row number, column number, and color channel.  The images were greyscale rather than color, so there is only 1 channel.

Finally, we divide the values by 255, so all the data is between 0 and 1.  This improves optimization with the default parameters for the Adam optimizer.

Then we can load in the CSV file with Pandas and pass it to our `data_prep` function. This gives us an array of predictors which I’ll call `x`, and an array of the target which I’ll call `y`.

In [ ]:
train_file = "../input/digit-recognizer/train.csv"
raw_data = pd.read_csv(train_file)

x, y = data_prep(raw_data)

Now we’ll build our model.  It looks like the model you built earlier, except the first layer is a `Conv2D` or 2D convolutional layer rather than a pretrained model. We also added another convolutional layer before the `Flatten` layer. I need to specify a few arguments for each convolutional layer.  The number of convolutions or filters to include in that layer, the size in pixels of the convolutions  (which is called the kernel size) and the activation function.  

For the very first layer, I need to specify the shape of each image, but I won’t need that for subsequent layers.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout

model = Sequential()
model.add(Conv2D(20, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(Conv2D(20, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In this case, I used 2 convolutional layers, though I could have used more if I’d wanted.  As you saw with transfer learning, I’ll use a flatten layer at some point to convert the output of previous layers into a 1D representation for each image.  These models usually perform better if you add a `Dense` layer in between the `Flatten` layer and the final prediction layer.  So I’ve added a dense layer with 128 nodes. Finally we have the output layer, the same as the output layers you’ve written before.

I compile it with the same arguments you’ve seen before.

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

Then I’ll fit it.  Because I have the whole data loaded into arrays already, I use the `fit` command, rather than `fit_generator` which we used with `ImageDataGenerator`.  

I supply the predictors `x` and the target `y`. As you’ve seen before, I specify a batch size and number of epochs.  Then finally, I want to get a validation score. I use the `validation_split` argument to say that 20% of the data should be set aside for validation, leaving 80% for training data.

In [ ]:
model.fit(x, y,
          batch_size=128,
          epochs=2,
          validation_split = 0.2)

When I run this, I can watch the accuracy go from 10% when I first start training to about 98% pretty quickly.

You can experiment with a few things to improve the model slightly.  You can change the number of convolutions in any layer.  You could add or remove layers, either adding convolutional layers before the flatten, or dense layers after it. Or you could change the filter or kernel size in any layer.

When you increase the number of layers or the number of convolutions in a layer, you are increasing the model’s ability to fit training data. We call that the model capacity.  That can improve a model that was underfitting, or it could cause you to overfit. Use validation scores as your ultimate measure of model quality as you experiment with different model architectures.

# Your Turn

Now it’s your turn to **[build your own model](#$EXERCISE_FORKING_URL$)** on the fashion data, building a model to detect clothing type. After that, I’ll show you a couple tricks to prevent overfitting and to make your models run faster.